In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from statsmodels.base.model import GenericLikelihoodModel

## Part 1

## Part 2

## Part 3

In [18]:
BETA = .9
GAMMA = .5772 #euler's constant

def value_helper(a, theta1, cost, v_init):
    """helper function for calculating value function.
    
    given the value for the first period, calculate forward 
    'a' periods"""
    if a <= 0: #we have reacahed the last period
        return  v_init
    
    else:
        v_next = value_helper(a-1, theta1, cost, v_init)
        sim_0 = a*theta1 + BETA*(GAMMA + np.log( np.exp(v_next[-1][0]) + np.exp(v_next[-1][1]) ))  
        sim_1 = cost + BETA*(GAMMA + np.log(  np.exp(v_next[-1][0]) + np.exp(v_next[-1][1]) )) 
        
        return np.concatenate( ( v_next,[[sim_0,sim_1]]) )


v = value_helper(5, -1, -3, [[0,0]])

In [17]:
def value_function(a, theta1, cost, v_init, error, maxiter):
    """solve for the first period of the value function
    with the contraction mapping loop
    
     You can choose how far into the future it goes by setting max_periods"""
    
    #only need to iterate 1 periods into the future
    v = value_helper(1, theta1, cost, v_init)
    
    #stop iterating when the last two periods look the same
    while ( maxiter >= 0  
           and ( abs(  v[1,0] - v[0,0] )  > error
           or abs(  v[1,1] - v[0,1] )  > error) ):
        
        #recompute value function until convergence
        v = value_loop(theta1, cost, [v[1,:]], error, maxiter-1)  
    
    return value_helper(a-1, theta1, cost, [v[1,:]])


print value_function(5, -1, -3, [[0,0]],.01, 100)

[[-3.5771012  -5.5771012 ]
 [-3.58567587 -5.58567587]
 [-4.59339307 -5.59339307]
 [-6.33263825 -6.33263825]
 [-8.55606196 -7.55606196]]


## Part 4

In [8]:
v = value_function(-1, -3, [[0,0]] , .01 , 2)
print v
print v.shape

#difference between e0 and e1
diff = v[1,0] - v[1,1]
print diff

#probability of this different
print np.exp(-diff)/(1+np.exp(-diff))

#PDV at = 4, e0= 1, e1=-1.5
print np.maximum(-3 + 1 + BETA*v[4,0], -4 + -1.5 +BETA*v[4,1] )

[[-10.9934144   -8.9934144 ]
 [ -8.54981102  -7.54981102]
 [ -6.32569276  -6.32569276]
 [ -4.58567587  -5.58567587]
 [ -3.5771012   -5.5771012 ]
 [ -3.56757379  -5.56757379]]
(6, 2)
-1.0
0.7310585786300049
-5.219391079298751


In [21]:
data = np.loadtxt("data.asc")
print data.shape

(1000, 2)


In [25]:
class Rust(GenericLikelihoodModel):
    """class for estimating the values of R and theta"""
    
    def nloglikeobs(self, params, v=False):
        
        theta1, R = params
        
        # Input our data into the model
        i = self.exog[:,0] #reshape
        a = self.endog.astype(int)
        
        #solve value function based on params
        v = value_function(5, theta1, R, [[0,0]] , .01 , 100)
        
        #interpolate using scipy (easier than indexing)
        v0 = interp1d(range(1,6), v[:,0],fill_value="extrapolate")
        v1 = interp1d(range(1,6), v[:,1],fill_value="extrapolate")
        
        diff = v1(a) - v0(a)
    
        #calculate likelihood of each obs
        pr0 = 1/(1+np.exp(diff))
        pr1 = np.exp(diff)/(1+np.exp(diff))

        likelihood = (1-i)*pr0 + i*pr1
        return -np.log(likelihood).sum()
    
    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        if start_params == None:
            start_params = [1,1]
        return super(Rust, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    
    
model = Rust(data[:,0],data[:,1])

result = model.fit()
print(result.summary(xname=['theta_1', 'R']))

Optimization terminated successfully.
         Current function value: 0.452716
         Iterations: 53
         Function evaluations: 104
                                 Rust Results                                 
Dep. Variable:                      y   Log-Likelihood:                -452.72
Model:                           Rust   AIC:                             907.4
Method:            Maximum Likelihood   BIC:                             912.3
Date:                Tue, 18 Dec 2018                                         
Time:                        10:35:51                                         
No. Observations:                1000                                         
Df Residuals:                     999                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------